# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo:27017")
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

In [4]:
df_youtube.columns[[0]]

Index(['video_id'], dtype='object')

### Netoyer les données

In [5]:
df_youtube = df_youtube.dropna(subset=['date'])
df_youtube.rename(columns={'video_id':'_id'}, inplace=True)
df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')
l_d = df_youtube.to_dict(orient='records')

In [10]:
import json

with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)
for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [11]:
from pymongo import MongoClient

client = MongoClient("mongo:27017")

db_yt = client.yt

collection = db_yt['videos']

collection.delete_many({})

collection.insert_many(l_d)

In [50]:
collection.find_one()

{'_id': '5ywKal6-anc',
 'title': 'Gigi Hadid Loses High Heel During Fashion Week, Walks It Off Like a Pro | TMZ',
 'channel_title': 'TMZ',
 'category_id': 24,
 'tags': 'TMZ2016FS11221|TMZ|Hollywood|Celebrity|Entertainment|Famous|Hollywood News|Fame|Entertainment News',
 'views': 703750,
 'likes': 2921,
 'dislikes': 2196,
 'comment_total': 1042,
 'thumbnail_link': 'https://i.ytimg.com/vi/5ywKal6-anc/default.jpg',
 'date': 13.09,
 'category_name': 'Entertainment'}

## Question 1  

1) Récupérer toutes les vidéos de la chaîne Apple.

In [36]:
list(collection.find({'channel_title':'Apple'}))

[{'_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 19707391,
  'likes': 381919,
  'dislikes': 69465,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 17.09},
 {'_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|airpods|apple watch|apple watch series 3|apple watch cellular|apple watch 4g|apple music|40 million songs on your wrist|apple skateboard ad|apple train station ad|apple roll ad|apple watch skateboard ad|apple watch train station ad|apple watch roll ad|airpods skateboard ad|airpods train station ad|airpods roll ad|apple music skateboard ad|apple music train station ad|apple music roll ad|misbehaving',


## Question 2

2) Compter le nombre de catégories différentes

In [63]:
list(collection.aggregate([
    {"$group" : {"_id" : "$category_id"}},
    {"$count": "total_category"}
]))

[{'total_category': 16}]

## Question 3

3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.

TypeError: index 'tags' cannot be applied to Cursor instances

## Question 4

4) Récupérer les vidéos les plus vues.

In [60]:
list(collection.find().sort([('views', -1)]).limit(3))

[{'_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09},
 {'_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': 'Dusk Till Dawn|Pop|RCA Records Label|ZAYN feat. Sia',
  'views': 36323498,
  'likes': 1431683,
  'dislikes': 28049,
  'comment_total': 100661,
  'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg',
  'date': 13.09},
 {'_id': 'LunHybOKIjU',
  'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'channel_title': 'BETNetworks',
  'category_id': 24,
  'tags': 'BET|BET Networks|BET Music|Black Entertainment Television|eminem|freestyle|full|hip hop awards|hip hop awards 2017|bet hip hop aw

## Question 5

5) Compter le nombre moyen de vues en fonction de la catégorie.

In [61]:
list(collection.aggregate([{"$group" : {"_id" : "$category_id", "averageView" : {"$avg" : "$views"}}}]))

[{'_id': 19, 'averageView': 426492.22222222225},
 {'_id': 1, 'averageView': 966547.2871287129},
 {'_id': 2, 'averageView': 622727.8648648649},
 {'_id': 22, 'averageView': 892992.28515625},
 {'_id': 43, 'averageView': 8726.0},
 {'_id': 27, 'averageView': 550585.2659574468},
 {'_id': 20, 'averageView': 777213.0357142857},
 {'_id': 24, 'averageView': 1059427.7735849055},
 {'_id': 28, 'averageView': 861563.6158940397},
 {'_id': 17, 'averageView': 618074.711409396},
 {'_id': 10, 'averageView': 1442240.943283582},
 {'_id': 29, 'averageView': 681178.2},
 {'_id': 25, 'averageView': 533575.1262626263},
 {'_id': 15, 'averageView': 633906.4210526316},
 {'_id': 23, 'averageView': 1317200.7289719626},
 {'_id': 26, 'averageView': 556002.6679389313}]

## Question 6 

6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [59]:
list(collection.aggregate([
    {"$group":{
        "_id" : "$channel_title", 
        "averageLike" : {"$avg" :"$likes"}}
    },
    {"$sort" : {"averageLike" : -1}},
    {"$limit" : 5}
]))

[{'_id': 'ibighit', 'averageLike': 1533457.0},
 {'_id': 'ZaynVEVO', 'averageLike': 1431683.0},
 {'_id': 'BETNetworks', 'averageLike': 982842.0},
 {'_id': 'melanie martinez', 'averageLike': 921577.0},
 {'_id': 'jypentertainment', 'averageLike': 793387.0}]